# Imports

In [1]:
#!/usr/bin/env python
# coding: utf-8

# ___
# 
# ___
# # Text Generation with Neural Networks
# 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

tf.__version__

'2.8.0'

# Step 1 the data

In [2]:
# ## Step 1: The Data
# 
# You can grab any free text you want from here: https://www.gutenberg.org/
# 
# We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:
# 
# 1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.
# 
# 2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

path_to_file = 'shakespeare.txt'
text = open(path_to_file, 'r').read()
print(text[:500])

# ### Understanding unique characters
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

# Step 2 Text Processing

In [3]:
# ## Step 2: Text Processing

# ### Text Vectorization
# 
# We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

char_to_ind = {u:i for i, u in enumerate(vocab)}
char_to_ind

ind_to_char = np.array(vocab)
ind_to_char

encoded_text = np.array([char_to_ind[c] for c in text])
encoded_text

# We now have a mapping we can use to go back and forth from characters to numerics.
sample = text[:20]
sample

encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Step 3 Creating Batches

In [4]:
# ## Step 3: Creating Batches
# 
# Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence is. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character?) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

print(text[:500])
line = "From fairest creatures we desire increase"
len(line)

part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""
len(part_stanza)


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


131

In [5]:
# ### Training Sequences
# 
# The actual text data will be the text sequence shifted one character forward. For example:
# 
# Sequence In: "Hello my nam"
# Sequence Out: "ello my name"
# 
# 
# We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.


seq_len = 120
total_num_seq = len(text)//(seq_len+1)
total_num_seq

45005

In [6]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])


# The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:
# 
# drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
#     whether the last batch should be dropped in the case it has fewer than
#     `batch_size` elements; the default behavior is not to drop the smaller
#     batch.
# 

sequences = char_dataset.batch(seq_len+1, drop_remainder=True)



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


In [7]:
# Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:
# 
# 1. Grab the input text sequence
# 2. Assign the target text sequence as the input text sequence shifted by one step forward
# 3. Group them together as a tuple

def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

dataset = sequences.map(create_seq_targets)

for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [8]:
# ### Generating training batches
# 
# Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

# Batch size
batch_size = 64

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 120), dtype=tf.int32, name=None), TensorSpec(shape=(64, 120), dtype=tf.int32, name=None))>

# Step 4 Creating the Model

In [9]:
# ## Step 4: Creating the Model

# This is where YOU will create the model. it needs to start with an embedding layer.
# 
# The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM or GRU usually leads to more realisitic results.

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension. Your choice.
embed_dim = 256

# Number of RNN units. Your choice. YOU MUST EXPERIMENT WITH THIS NUMBER.
rnn_neurons = 512

# Value of the Dropout
dropout_value = 0.2

In [10]:
# Now let's create a function that easily adapts to different variables as shown above.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

from tensorflow.keras.initializers import GlorotNormal


# ### Setting up Loss Function
# 
# For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True
from tensorflow.keras.losses import sparse_categorical_crossentropy

# The reason we need to redefine this is to make sure we are using one hot encoding (from_logits=True)
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
# define your model here...
# don't forget you need an embeddings layer at the beginning
# and a dense layer the size of the vocabulary at the end to generate distributions 
    model = Sequential()
    model.add(Embedding(input_dim = vocab_size, output_dim = embed_dim, batch_input_shape=[batch_size, None]))
    #model.add(GRU(rnn_neurons,
    #        return_sequences=True,
    #        stateful=True,
    #        recurrent_initializer=GlorotNormal()
    #            ))
    model.add(LSTM(rnn_neurons,
                   return_sequences=True,
                   stateful=True,
                   recurrent_initializer=GlorotNormal()
                  ))
    model.add(Dropout(dropout_value))
    model.add(Dense(vocab_size))
    return model
 
# create your model here, passing the parameters
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           21504     
                                                                 
 lstm (LSTM)                 (64, None, 512)           1574912   
                                                                 
 dropout (Dropout)           (64, None, 512)           0         
                                                                 
 dense (Dense)               (64, None, 84)            43092     
                                                                 
Total params: 1,639,508
Trainable params: 1,639,508
Non-trainable params: 0
_________________________________________________________________


## Test model without training it

In [12]:
for input_example_batch, target_example_batch in dataset.take(1):

    
    # Predict off some random batch
    example_batch_predictions = model(input_example_batch)

    # Display the dimensions of the predictions
    print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


example_batch_predictions

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices

# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))

(64, 120, 84)  <=== (batch_size, sequence_length, vocab_size)
Given the input seq: 

  bowls I'll win to-night of him. Come, go.
  SECOND LORD. I'll attend your lordship.
                                  


Next Char Predictions: 

B:d]`s}?CL[B)8E`T yBX;z"0HmUnjWIfj|}J>WaTtZ3ULeQ|_[os]59t_fix?d<V:M!l)ndH0>n:E0uB<-0-W1!x7f<D5cHl_igXAh4;lK)QH3
a7[>) a,


## Train the model

In [13]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy 

In [14]:
model.compile(loss = sparse_cat_loss, optimizer = Adam(learning_rate=0.01), metrics=['accuracy'])

r = model.fit(dataset, epochs = 30)

In [ ]:
plt.plot(r.history['loss'], label='loss')

plt.title('Training error ' + str(rnn_neurons) + ' rnn neurons')
plt.xlabel('epoch')
plt.ylabel('error')
plt.legend()
plt.show()

plt.plot(r.history['accuracy'], label='accuracy')

plt.title('Training accuracy ' + str(rnn_neurons) + ' rnn neurons')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.show()

# Save the model

In [16]:
model_name = 'shakespeare_gen_.h5'
model.save(model_name ,save_format='h5')

# Generate text

In [28]:
from tensorflow.keras.models import load_model

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights(model_name)
model.build(tf.TensorShape([1, None]))
model.summary()

def generate_text(model, start_seed,gen_size=100,temp=1):
      
      #model: Trained Model to Generate Text
      #start_seed: Intial Seed text in string form
      #gen_size: Number of characters to generate

      #Basic idea behind this function is to take in some seed text, format it so
      #that it is in the correct shape for our network, then loop the sequence as
      #we keep adding our own predicted characters. Similar to our work in the RNN
      #time series problems.
      
    # Number of characters to generate
    num_generate = gen_size

    # Vecotrizing starting seed text
    input_eval = [char_to_ind[s] for s in start_seed]

    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to hold resulting generated text
    text_generated = []

    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Higher probability == lesss surprising/ more expected
    # Lower temperature == more surprising / less expected
 
    temperature = temp

    # Here batch size == 1
    model.reset_states()

    for i in range(num_generate):

      # Generate Predictions
        predictions = model(input_eval)

      # Remove the batch shape dimension
        predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
        input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

print(generate_text(model,"spain",gen_size=400))
# Note: "JULIET" is used as a seed. Experiment with other words to get different results.
# You can also vary the size of the generated string.

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (1, None, 256)            21504     
                                                                 
 lstm_12 (LSTM)              (1, None, 512)            1574912   
                                                                 
 dropout_12 (Dropout)        (1, None, 512)            0         
                                                                 
 dense_12 (Dense)            (1, None, 84)             43092     
                                                                 
Total params: 1,639,508
Trainable params: 1,639,508
Non-trainable params: 0
_________________________________________________________________
spain.
    For the thing carry?
  AMILIBANUS. Say I thank not the beauty.
  LUCIUS. The man.                                                                                   